In [5]:
import numpy as np
import multiprocessing as mp
import os
import cv2
from scipy import ndimage
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tnrange
from keras.models import Sequential, Model
from tensorflow.keras.models import load_model

In [2]:
# Para facilitar la ejecución del codigo se descargan directamente los datos y se descomprimen los archivos.

!wget https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip
!wget https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip

!unzip -qq images_background.zip
!unzip -qq images_evaluation.zip

--2023-09-19 22:33:47--  https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/brendenlake/omniglot/master/python/images_evaluation.zip [following]
--2023-09-19 22:33:47--  https://raw.githubusercontent.com/brendenlake/omniglot/master/python/images_evaluation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6462886 (6.2M) [application/zip]
Saving to: ‘images_evaluation.zip’

images_evaluation.z 100%[===================>]   6.16M  --.-KB/s    in 0.09s   

2023-09-19 22:33:48 (70.5 MB/s) - ‘images_evaluation.zip’ saved [6

In [8]:
def lee_alfabetos(ruta_al_directorio_del_alfabeto, nombre_del_alfabeto):
    """
    Lee todos los caracteres de los alfabetos que contenga el directorio dado.

    Argumentos:
    - ruta_al_directorio_del_alfabeto: Ruta del directorio que contiene el alfabeto.
    - nombre_del_alfabeto: Nombre del alfabeto.

    Devuelve:
    - datax: Lista de imágenes.
    - datay: Lista de etiquetas asociadas a las imágenes.
    """
    datax = []
    datay = []

    # Listar todos los caracteres en el directorio del alfabeto
    caracteres = os.listdir(ruta_al_directorio_del_alfabeto)

    for caracter in caracteres:
        # Listar todas las imágenes dentro del directorio del caracter
        imagenes = os.listdir(ruta_al_directorio_del_alfabeto + caracter + '/')
        for im in imagenes:
            # Leer y redimensionar la imagen en escala de grises
            imagen = cv2.resize(
                cv2.imread(ruta_al_directorio_del_alfabeto + caracter + '/' + im, cv2.IMREAD_GRAYSCALE),
                (28, 28)
            )

            # Crear rotaciones de la imagen (90, 180 y 270 grados)
            rotada_90 = ndimage.rotate(imagen, 90)
            rotada_180 = ndimage.rotate(imagen, 180)
            rotada_270 = ndimage.rotate(imagen, 270)

            # Agregar las imágenes y sus rotaciones al conjunto de datos
            datax.extend((imagen, rotada_90, rotada_180, rotada_270))

            # Agregar etiquetas para las imágenes y sus rotaciones
            datay.extend((
                ruta_al_directorio_del_alfabeto + '_' + caracter + '_0',
                ruta_al_directorio_del_alfabeto + '_' + caracter + '_90',
                ruta_al_directorio_del_alfabeto + '_' + caracter + '_180',
                ruta_al_directorio_del_alfabeto + '_' + caracter + '_270'
            ))

    return np.array(datax), np.array(datay)

def preproces_omniglot(directorio_principal):
    """
    Llama a la función lee_alfabetos para leer cada uno de los alfabetos del directorio principal.

    Argumentos:
    - directorio_principal: Ruta del directorio que contiene varios alfabetos.

    Devuelve:
    - datax: Lista consolidada de todas las imágenes.
    - datay: Lista consolidada de todas las etiquetas.
    """
    datax = None
    datay = None

    # Utilizar múltiples núcleos del procesador para procesar los alfabetos en paralelo
    pool = mp.Pool(mp.cpu_count())

    # Leer cada directorio (alfabeto) en el directorio principal
    results = [pool.apply(lee_alfabetos,
                          args=(
                              directorio_principal + '/' + directorio + '/', directorio,
                          )) for directorio in os.listdir(directorio_principal)]
    pool.close()

    # Consolidar los datos de todos los alfabetos
    for resul in results:
        if datax is None:
            datax = resul[0]
            datay = resul[1]
        else:
            datax = np.vstack([datax, resul[0]])
            datay = np.concatenate([datay, resul[1]])

    return datax, datay


In [9]:
# Llama a la función preproces_omniglot para preprocesar las imágenes en el directorio 'images_background' y
# almacena las imágenes y sus etiquetas en trainx y trainy, respectivamente.
trainx, trainy = preproces_omniglot('images_background')

# Llama a la función preproces_omniglot para preprocesar las imágenes en el directorio 'images_evaluation' y
# almacena las imágenes y sus etiquetas en testx y testy, respectivamente.
testx, testy = preproces_omniglot('images_evaluation')

# Devuelve las dimensiones (shapes) de los conjuntos de datos trainx, trainy, testx y testy.
# Esto es útil para entender rápidamente el número de ejemplos y las dimensiones de las imágenes en cada conjunto.
trainx.shape, trainy.shape, testx.shape, testy.shape


((77120, 28, 28), (77120,), (52720, 28, 28), (52720,))

In [20]:
def crea_muestra(n_way, n_support, n_query, datax, datay):
    """
    Crea una muestra aleatoria de tamaño n_support+n_query, de imágenes de n_way clases.

    Entrada:
        n_way (int): Número de clases.
        n_support (int): Número de imágenes en el conjunto soporte por clase.
        n_query (int): Número de imágenes para clasificar por clase.
        datax (np.array): Conjunto de imágenes.
        datay (np.array): Conjunto de etiquetas.

    Salida:
        (dict) de:
            (tf.Tensor): Muestra de imágenes de tamaño (n_way, n_support+n_query, (dim)).
            (int): n_way.
            (int): n_support.
            (int): n_query.
    """

    # Obtiene las clases únicas en el conjunto de etiquetas
    clases_unicas = np.unique(datay)

    # Elige n_way clases de manera aleatoria sin repetición
    clases_elegidas = np.random.choice(clases_unicas, n_way, replace=False)

    # Total de ejemplos por clase
    ejemplos_por_clase = n_support + n_query
    # Crea una lista de índices para todo el conjunto de etiquetas
    indices = np.arange(len(datay))

    muestra = []

    for clase in clases_elegidas:
        # Encuentra los índices de las imágenes que pertenecen a la clase actual
        clase_indices = indices[datay == clase]

        # Elige aleatoriamente ejemplos_por_clase índices de la clase actual
        indices_elegidos = np.random.choice(clase_indices, ejemplos_por_clase, replace=False)

        # Agrega las imágenes elegidas a la muestra
        muestra.append(datax[indices_elegidos])

    # Convierte la lista de imágenes en un tensor de TensorFlow
    muestra = tf.convert_to_tensor(np.array(muestra), dtype=tf.float32)

    # Devuelve un diccionario con la muestra y otros metadatos
    return {
        'imagenes': muestra,
        'n_way': n_way,
        'n_support': n_support,
        'n_query': n_query
    }


In [11]:
def red_convolucional(tamaño_entrada, dimension_caracteristicas):
    """
    Define y retorna una red convolucional que se usará para extraer características de las imágenes.

    Entrada:
        tamaño_entrada (tuple): Dimensiones de la imagen de entrada (alto, ancho, canales).
        dimension_caracteristicas (int): Número de características a extraer con la última capa densa.

    Salida:
        model (tf.keras.Sequential): Modelo de red convolucional.
    """

    # Inicializa un modelo secuencial en Keras.
    model = tf.keras.Sequential()

    # Primera capa convolucional.
    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=tamaño_entrada))  # Convolucional con 64 filtros y kernel de 3x3.
    model.add(layers.BatchNormalization())  # Normalización por lotes para estabilizar y acelerar el entrenamiento.
    model.add(layers.ReLU())  # Función de activación ReLU.
    model.add(layers.MaxPooling2D((2, 2)))  # Reducción de dimensiones mediante MaxPooling.

    # Añade tres bloques idénticos de capas convolucionales, cada bloque consiste en una capa convolucional,
    # una capa de normalización por lotes, una activación ReLU y una capa de MaxPooling.
    for _ in range(3):
        model.add(layers.Conv2D(64, (3, 3), padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ReLU())
        model.add(layers.MaxPooling2D((2, 2)))

    # Añade una capa que aplana la salida anterior para prepararla para la capa densa.
    model.add(layers.Flatten())
    # Añade una capa densa que genera un vector de características con una longitud igual a dimension_caracteristicas.
    model.add(layers.Dense(dimension_caracteristicas))

    return model  # Retorna el modelo construido.


In [12]:
def euclidean_dist(x, y):
    """
    Calcula la distancia euclidea al cuadrado entre x e y.

    Entrada:
        x (tf.Tensor): tamaño (n, d). n es n_way*n_query.
        y (tf.Tensor): tamaño (m, d). m es n_way.

    Salida:
        tf.Tensor: tamaño (n, m). Para cada imagen de consulta, las distancias a cada uno de los prototipos.
    """
    # Obtiene las dimensiones de x e y.
    n = tf.shape(x)[0]  # Número de filas en x.
    m = tf.shape(y)[0]  # Número de filas en y.
    d = tf.shape(x)[1]  # Número de columnas (características) en x (o y, ya que ambos tienen la misma dimensión en columnas).

    # Expande las dimensiones de x e y para hacerlos broadcastable.
    # x pasa de (n, d) a (n, 1, d).
    x = tf.expand_dims(x, 1)
    # Repite x 'm' veces en la segunda dimensión para que tenga tamaño (n, m, d).
    x = tf.tile(x, (1, m, 1))

    # y pasa de (m, d) a (1, m, d).
    y = tf.expand_dims(y, 0)
    # Repite y 'n' veces en la primera dimensión para que tenga tamaño (n, m, d).
    y = tf.tile(y, (n, 1, 1))

    # Calcula la diferencia al cuadrado entre x e y, y la suma a lo largo del eje de las características (axis=2).
    # El resultado es una matriz de tamaño (n, m) donde cada entrada (i, j) representa la distancia euclidiana al cuadrado
    # entre el i-ésimo vector en x y el j-ésimo vector en y.
    return tf.reduce_sum(tf.square(x - y), axis=2)


In [13]:
def floss(y_true, y_pred):
    """
    Calcula la función de pérdida personalizada para dos vectores: uno que contiene las etiquetas verdaderas y
    otro que contiene las probabilidades predichas de las clases.

    Entrada:
        y_true (tf.Tensor): Tensor que contiene las etiquetas verdaderas en formato one-hot encoding.
        y_pred (tf.Tensor): Tensor que contiene las probabilidades predichas de las clases.

    Salida:
        tf.Tensor: Un valor escalar que representa la pérdida calculada.
    """

    # Multiplica elemento a elemento las etiquetas verdaderas con las probabilidades predichas
    # Luego, sumamos esos valores a lo largo del último eje (axis=-1)
    # Finalmente, calculamos el negativo de la media de esos valores sumados

    loss_val = -tf.reduce_mean(tf.reduce_sum(y_true * y_pred, axis=-1))

    return loss_val

In [14]:
def accuracyProt(y_true, y_pred, n_way, n_query):
    """
    Calcula la precisión de las predicciones basándose en etiquetas verdaderas y predicciones.

    Entrada:
        y_true (tf.Tensor): Tensor que contiene las etiquetas verdaderas en formato one-hot encoding.
        y_pred (tf.Tensor): Tensor que contiene las probabilidades predichas de las clases.
        n_way (int): Número de clases únicas.
        n_query (int): Número de imágenes para clasificar por clase.

    Salida:
        float: Precisión de las predicciones.
    """

    # Encuentra el índice del valor máximo en y_pred a lo largo del eje 1
    # Esto nos dará la clase predicha para cada muestra.
    predicciones = tf.argmax(y_pred, axis=1)

    # Encuentra el índice del valor máximo en y_true a lo largo del eje 1
    # Esto nos dará la etiqueta verdadera para cada muestra.
    etiquetas_correctas = tf.argmax(y_true, axis=1)

    # Compara las predicciones con las etiquetas correctas y cuenta cuántas coinciden.
    total_correct = tf.reduce_sum(tf.cast(tf.equal(predicciones, etiquetas_correctas), tf.int32))

    # Calcula la precisión dividiendo el número total de predicciones correctas
    # por el número total de predicciones (n_way * n_query).
    accuracyp = total_correct / (n_way * n_query)

    return accuracyp

In [15]:
def Proto(sample, n_way, n_support, n_query, model):
    """
    Implementa el Prototipo de redes para el aprendizaje de pocas tomas.

    Entrada:
        sample (dict): Diccionario que contiene las imágenes.
        n_way (int): Número de clases únicas.
        n_support (int): Número de imágenes de soporte por clase.
        n_query (int): Número de imágenes de consulta por clase.
        model (tf.keras.Model): Modelo de red neuronal para extraer características.

    Salida:
        loss (tf.Tensor): Valor de la función de pérdida calculada.
        target_inds_onehot (tf.Tensor): Etiquetas verdaderas en formato one-hot encoding.
        log_p_y (tf.Tensor): Log-probabilidades predichas.
    """

    # Separa las imágenes de soporte y consulta del diccionario 'sample'
    x_support = sample['imagenes'][:, :n_support]
    x_query = sample['imagenes'][:, n_support:]

    # Reajusta las dimensiones de las imágenes de soporte y consulta
    x_support = np.reshape(x_support, (x_support.shape[0] * x_support.shape[1], *x_support.shape[2:]))
    x_query = np.reshape(x_query, (x_query.shape[0] * x_query.shape[1], *x_query.shape[2:]))

    # Convierte las imágenes a tensores y normaliza sus valores (0-255 a 0-1)
    x_support = tf.convert_to_tensor(x_support / 255.0, dtype=tf.float32)
    x_query = tf.convert_to_tensor(x_query / 255.0, dtype=tf.float32)

    # Genera las etiquetas verdaderas para las imágenes de consulta en formato one-hot
    target_inds = [i // n_query for i in range(n_way * n_query)]
    target_inds_onehot = tf.keras.utils.to_categorical(target_inds, num_classes=n_way)
    target_inds_onehot = tf.convert_to_tensor(target_inds_onehot)

    # Usa el modelo para extraer las características de las imágenes de soporte y consulta
    z_support = model(x_support)
    z_query = model(x_query)

    # Calcula el prototipo (representación promedio) para cada clase a partir de las características de soporte
    z_proto = tf.reduce_mean(tf.reshape(z_support, (n_way, n_support, -1)), axis=1)

    # Calcula la distancia euclídea entre las características de consulta y los prototipos
    dists = euclidean_dist(z_query, z_proto)

    # Calcula las log-probabilidades predichas a partir de las distancias
    log_p_y = tf.nn.log_softmax(-dists, axis=1)

    # Calcula la función de pérdida usando las etiquetas verdaderas y las log-probabilidades predichas
    loss = floss(target_inds_onehot, log_p_y)

    return loss, target_inds_onehot, log_p_y


In [16]:
def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epocas, tamaño_epoca):
    """
    Entrena el modelo de una red prototípica utilizando aprendizaje de pocos disparos.

    Argumentos de entrada:
    - model (tf.keras.Model): Modelo prototípico basado en una red convolucional.
    - optimizer (tf.keras.optimizers.Optimizer): Optimizador utilizado para actualizar los pesos del modelo.
    - train_x (np.array): Conjunto de imágenes de entrenamiento.
    - train_y (np.array): Etiquetas correspondientes a las imágenes de entrenamiento.
    - n_way (int): Número de clases distintas a considerar en cada episodio de aprendizaje.
    - n_support (int): Número de imágenes por clase que se usan como ejemplos de soporte en cada episodio.
    - n_query (int): Número de imágenes por clase que se usan como ejemplos de consulta en cada episodio.
    - max_epocas (int): Número máximo de épocas de entrenamiento.
    - tamaño_epoca (int): Número de episodios por época.

    Salida:
    - epoca_acc (float): Precisión del modelo en la última época de entrenamiento.
    """
    # Programa un decaimiento exponencial para la tasa de aprendizaje del optimizador.
    scheduler = tf.keras.optimizers.schedules.ExponentialDecay(0.001, decay_steps=2000, decay_rate=0.5, staircase=True)

    epoca = 0  # Contador de épocas realizadas
    stop = False  # Condición para detener el entrenamiento, actualmente no se utiliza
    accuracies = []  # Lista para almacenar las precisión de cada época

    # Bucle de entrenamiento principal
    while epoca < max_epocas and not stop:
        running_loss = 0.0  # Acumulador de la pérdida durante la época
        running_acc = 0.0  # Acumulador de la precisión durante la época

        # Bucle para cada episodio dentro de una época
        for episode in tnrange(tamaño_epoca, desc="Epoca {:d} Entrenamiento".format(epoca + 1)):
            tf.keras.backend.clear_session()  # Borra el gráfico anterior y previene el aumento de memoria

            # Permite el seguimiento automático de las operaciones para calcular gradientes más adelante
            with tf.GradientTape() as tape:
                # Crea una muestra de entrenamiento para el episodio actual
                muestra = crea_muestra(n_way, n_support, n_query, train_x, train_y)

                # Calcula la pérdida usando el modelo prototípico
                loss, target_inds_onehot, log_p_y  = Proto(muestra, n_way, n_support, n_query, model)

            # Calcula los gradientes respecto a la pérdida
            gradients = tape.gradient(loss, model.trainable_variables)
            # Aplica los gradientes al modelo
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # Calcula la precisión del episodio
            accuracy = accuracyProt(target_inds_onehot, log_p_y, n_way, n_query)

            # Actualiza los acumuladores
            running_loss += loss
            running_acc += accuracy

        # Calcula las métricas promedio de la época
        epoca_loss = running_loss / tamaño_epoca
        epoca_acc = running_acc / tamaño_epoca

        # Muestra las métricas de la época
        print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoca + 1, epoca_loss, epoca_acc))

        epoca += 1  # Incrementa el contador de épocas

    return epoca_acc  # Devuelve la precisión de la última época




In [17]:
def create_model():
    """
    Crea un modelo de red neuronal convolucional para procesar imágenes de 28x28 píxeles en escala de grises.

    Devoluciones:
        tf.keras.Model: Modelo convolucional con una salida en un espacio de características de dimensión 64.
    """
    return red_convolucional((28, 28, 1), 64)


In [18]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 64)        256       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 64)        2

In [27]:
# Define parámetros del modelo y del entrenamiento
n_ways = [60]  # Lista de diferentes "n_way" a considerar (en este caso, solo 60 clases)
n_support = 5  # Número de imágenes de soporte por clase
n_query = 5    # Número de imágenes de consulta por clase

# Asigna los datos de entrenamiento (previamente preprocesados)
train_x = trainx
train_y = trainy

# Define parámetros para el proceso de entrenamiento
max_epoch = 5       # Número máximo de épocas de entrenamiento
epoch_size = 2000   # Número de episodios por época

# Itera sobre cada "n_way" en la lista n_ways
for n_way in n_ways:
    # Define un nombre único para el modelo basado en "n_way" y "n_support"
    model_name = f'model{n_way}w{n_support}s'

    # Crea un nuevo modelo y define el optimizador Adam
    model = create_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Entrena el modelo con los parámetros actuales
    accuracy = train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size)

    # Guarda el modelo entrenado en el disco
    model.save(f'{model_name}.h5')

    # Imprime información sobre el modelo entrenado
    print(f'Model {model_name} saved with accuracy {accuracy}')


In [22]:
def test(model, test_x, test_y, n_way, n_support, n_query, episodio_test):
    """
    Prueba el rendimiento del modelo entrenado en tareas de clasificación prototípica.

    Args:
        model: Modelo previamente entrenado.
        test_x (np.array): Conjunto de imágenes para prueba.
        test_y (np.array): Etiquetas asociadas a las imágenes de prueba.
        n_way (int): Número de clases diferentes consideradas en cada tarea de clasificación.
        n_support (int): Número de imágenes de soporte por clase.
        n_query (int): Número de imágenes de consulta por clase que el modelo deberá clasificar.
        episodio_test (int): Número total de episodios/tareas que se ejecutarán para la prueba.

    Returns:
        float: Precisión promedio obtenida en todos los episodios de prueba.
    """

    # Inicializa las métricas que almacenarán la suma de las pérdidas y precisiones a lo largo de los episodios.
    running_loss = 0.0
    running_acc = 0.0

    # Itera a través de cada episodio de prueba.
    for episodio in tnrange(episodio_test):
        # Genera una nueva tarea de clasificación con clases y ejemplos aleatorios.
        muestra = crea_muestra(n_way, n_support, n_query, test_x, test_y)

        # Usa la función Proto para obtener la pérdida y las predicciones del modelo.
        loss, target_inds_onehot, log_p_y  = Proto(muestra, n_way, n_support, n_query, model)

        # Calcula la precisión del modelo en el episodio actual.
        accuracy = accuracyProt(target_inds_onehot, log_p_y, n_way, n_query)

        # Actualiza las métricas acumulativas.
        running_loss += loss
        running_acc += accuracy

    # Calcula las métricas promedio.
    avg_loss = running_loss / episodio_test
    avg_acc = running_acc / episodio_test

    # Imprime las métricas promedio.
    print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

    return avg_acc


In [26]:
# Definición de parámetros para pruebas.
n_ways_test = [5]      # Número de clases en las tareas de clasificación. (El codigo se deja preparado para hacer más de un n-way a la vez)
n_support = 5          # Número de imágenes de soporte por clase.
n_query = 5            # Número de imágenes de consulta por clase.
test_episode = 1000    # Número total de episodios/tareas de prueba.

# Lista para almacenar las precisiones de los modelos durante las pruebas.
test_accuracies = []

# Itera a través de cada valor en 'n_ways_test' para probar el modelo en diferentes configuraciones de clases.
for n_way in n_ways_test:
    # Nombre del modelo que será cargado. En este caso, se espera que el modelo 'model60w5sOmniglot.h5' exista.
    model_name = f'model60w5sOmniglot.h5'

    # Carga el modelo previamente entrenado.
    model = load_model(model_name)

    # Usa la función 'test' previamente definida para evaluar el rendimiento del modelo en la tarea de clasificación prototípica.
    accuracy = test(model, testx, testy, n_way, n_support, n_query, test_episode)

    # Agrega la precisión obtenida a la lista 'test_accuracies'.
    test_accuracies.append(accuracy)

    # Imprime el resultado de la prueba para este modelo.
    print(f'Model {model_name} tested with accuracy {accuracy}')
